In [23]:
print("Running")

Running


In [24]:
%pwd

'c:\\Users\\adegb\\Desktop\\Computer Vision Projects\\End-to-end-Medical-Chatbot-Implementation-using-Llama2\\research'

In [44]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
from langchain_pinecone import Pinecone

from pinecone import Pinecone as PP
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [26]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','7d216b7d-95c8-40cf-b28c-0df3650fedc3')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','gcp-starter')
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY 

Extract data from PDFs

In [27]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

load data from PDFs

In [28]:
extracted_data = load_pdf("../data/")

In [29]:
#extracted_data

Create text chunks

In [30]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

text chunks generation

In [31]:
text_chunks = text_split(extracted_data)
print("chunk length:", len(text_chunks))

chunk length: 7020


Create embedding model function

In [32]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

#download embedding model

In [33]:
embeddings = download_hugging_face_embeddings()

In [34]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [35]:
query_result = embeddings.embed_query("Query Test")
print("Length", len(query_result))

Length 384


In [36]:
#query_result

Initializing the Pinecone

In [45]:
pc = PP(api_key=PINECONE_API_KEY)
index_name="medicals-chatbot"
#Creating Embeddings for Each of The Text Chunks & storing
docsearch =Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [39]:
#If already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [40]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [41]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
def retrieval_qa_chain(llm, prompt, db):
    qa_chain= RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever = docsearch.as_retriever(search_kwargs={'k':2}),
        return_source_document = True,
        chain_type_kwargs=chain_type_kwargs
    )
    
    return qa_chain

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=retrieval_qa_chain({"query": user_input})
    print("Response : ", result["result"])

TypeError: retrieval_qa_chain() missing 2 required positional arguments: 'prompt' and 'db'

In [ ]:
# RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)